In [1]:
# ─── PARSE INPUT ────────────────────────────────────────────────────────────────

import re
from itertools import product
from functools import lru_cache

with open('input19') as file:
    lines = file.read().splitlines()

# get a list containing every rule, sorted by rule id
rule_ids = []
rules_sorted = []
string_list = []
for line in lines:
    match = re.match('(\d+):(.+)', line)
    if match:
        rule_ids.append(match.group(1))
        rules_sorted.append(line)
    match = re.match('[a-z]+', line)
    if match:
        string_list.append(match[0])

rule_list = list(zip(map(int, rule_ids), rules_sorted))
rule_list.sort()
rule_list = [rule for _, rule in rule_list]

In [2]:
# ─── CHALLENGE 1 ────────────────────────────────────────────────────────────────

rules_sorted = []
for line in rule_list:
    parsed_inner = ['AND']
    match = re.search('(:)(.*)', line)
    if not match:
        continue
    end = match.group(2)
    matches = re.findall('[\d\w|]+', end)
    
    parsed_inner_inner = []
    for item in matches:
        # is letter
        if re.match('[a-zA-Z]', item):
            parsed_inner = ['CHR', item]
            break
        
        # is number
        if re.match('\d', item):
            parsed_inner_inner.append(int(item))
            continue
        
        # is or
        if re.match('|', item):
            parsed_inner[0] = 'OR'
            parsed_inner.append(parsed_inner_inner)
            parsed_inner_inner = []
            continue
        
        assert False  # catch bugs
    
    if parsed_inner[0] in ['OR', 'AND']:
        parsed_inner.append(parsed_inner_inner)
        parsed_inner = parsed_inner
    rules_sorted.append(parsed_inner)


@lru_cache
def parse_one_rule_id(rule_id):
    rule = rules_sorted[rule_id]
    return parse_one_rule_list(rule, idx=rule_id)


def parse_one_rule_list(rule_list, idx=0):
    rule = rule_list

    if rule[0] == 'CHR':
        result = [rule[1]]
        return result

    if rule[0] == 'AND':
        new_list = [parse_one_rule_id(idx) for idx in rule[1]]
        result = [''.join(p) for p in product(*new_list)]
        return result

    if rule[0] == 'OR':
        result_sum = []
        for rule_list in rule[1:]:
            new_rule_list = ['AND', rule_list]
            result = parse_one_rule_list(new_rule_list)
            result_sum += result
        return result_sum


block_len_dict = {8:  len(parse_one_rule_id(8)[0]),
                  11: len(parse_one_rule_id(11)[0])}

block_dict = {8:  set(parse_one_rule_id(8)),
              11: set(parse_one_rule_id(11))}

def check_string(string, block_id):
    string_in_rule = False
    length = block_len_dict[block_id]
    string_front = string[:length]
    string_back = string[length:]
    block = block_dict[block_id]
    if string_front in block:
        string_in_rule = True
    return string_in_rule, string_back
    
counter = 0
for string in string_list:
    string_in_rule = True    
    if string_in_rule:
        string_in_rule, string = check_string(string, 8)
    if string_in_rule:
        string_in_rule, string = check_string(string, 11)
    if len(string) != 0:
        string_in_rule = False
    if string_in_rule:
        counter += 1 

print('result 1: ', counter)

result 1:  134


In [3]:
# ─── CHALLENGE 2 ────────────────────────────────────────────────────────────────

block_len_dict = {8:  len(parse_one_rule_id(8)[0]),
                  11: len(parse_one_rule_id(11)[0]),
                  42: len(parse_one_rule_id(42)[0]),
                  31: len(parse_one_rule_id(31)[0]),}

block_dict = {8:  set(parse_one_rule_id(8)),
              11: set(parse_one_rule_id(11)),
              42: set(parse_one_rule_id(42)),
              31: set(parse_one_rule_id(31))}


def check_string(string, block_id):
    string_in_rule = False
    length = block_len_dict[block_id]
    string_front = string[:length]
    string_back = string[length:]
    block = block_dict[block_id]
    if string_front in block:
        string_in_rule = True
    return string_in_rule, string_back


counter = 0
for string_source in string_list:
    any_match = False
    for n in range(10):
        string = string_source
        string_in_rule = True  # turns False, if any condition is not met

        for _ in range(n):
            result_true, string = check_string(string, 42)
            if not result_true:
                string_in_rule = False
                
            # if string is exhausted, no match
            if not len(string) > 0:
                string_in_rule = False

        counter_m = 0
        while len(string) > 0:
            counter_m += 1
            result_true, string = check_string(string, 31)
            if not result_true:
                string_in_rule = False
                
        if counter_m >= n:
            string_in_rule = False

        if string_in_rule:
            any_match = True
            break 

    if any_match:
        counter += 1 

print('result 2: ', counter)

result 2:  377
